In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=c7df7cacd3e0ab8b94c410e7e503af4b80432371d10aa199db20517ae2c806fe
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [4]:
from sentence_transformers import SentenceTransformer

question_encoder_model = "sentence-transformers/facebook-dpr-question_encoder-single-nq-base"
question_encoder = SentenceTransformer(question_encoder_model)

In [5]:
context_encoder_model = "sentence-transformers/facebook-dpr-ctx_encoder-single-nq-base"
context_encoder = SentenceTransformer(context_encoder_model)

In [6]:
folder_path = '/content/drive/MyDrive/Colab Notebooks/Metabuddy/Data Process/week2'

In [7]:
train_file_path = f'{folder_path}/train.txt'
valid_file_path = f'{folder_path}/valid.txt'
test_file_path = f'{folder_path}/test.txt'

In [10]:
def createTrainDataset(file_path):
  dataset = {'text' : [], 'summary' : []}
  start_tag = "<Text>"
  end_tag = "<Summary>"
  with open(file_path, "r") as file:
    for sentence in file:
      if start_tag not in sentence: continue
      start_index = sentence.find(start_tag) + len(start_tag)
      end_index = sentence.find(end_tag, start_index)

      text = sentence[start_index:end_index].strip()
      summary = sentence[end_index + len(end_tag):].strip()
      if summary == '[]' or summary == '':
          continue
      dataset['text'].append(text)
      dataset['summary'].append(summary)

  return dataset

In [11]:
def createDataset(file_path):
  dataset = {'text' : [], 'summary' : []}
  start_tag = "<Text>"
  end_tag = "<Summary>"
  with open(file_path, "r") as file:
    for sentence in file:
      if start_tag not in sentence: continue
      start_index = sentence.find(start_tag) + len(start_tag)
      end_index = sentence.find(end_tag, start_index)

      text = sentence[start_index:end_index].strip()
      summary = sentence[end_index + len(end_tag):].strip()
      dataset['text'].append(text)
      dataset['summary'].append(summary)

  return dataset

In [12]:
msc_train_dataset = createTrainDataset(train_file_path)
msc_valid_dataset = createDataset(valid_file_path)
msc_test_dataset = createDataset(test_file_path)

In [13]:
def process_noise(dataset):
    for i, data in enumerate(dataset):
        if data == '[]':
            dataset[i] = ''
    return dataset

In [ ]:
process_noise(msc_valid_dataset['summary'])
process_noise(msc_test_dataset['summary'])

In [15]:
question = 'Do I like beer?'

In [20]:
question_embedding = question_encoder.encode([question])
print(question_embedding.shape)

(1, 768)


In [40]:
context = msc_test_dataset['summary']

In [24]:
context_embedding = context_encoder.encode(context)
print(context_embedding.shape)



(24375, 768)


In [41]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_scores = cosine_similarity(question_embedding, context_embedding)[0]
top_indices = similarity_scores.argsort()[-3:][::-1]
top_contexts = [context[idx] for idx in top_indices]
print("Top 3 similar context sentences:")
for idx, cxt in zip(top_indices, top_contexts):
    score = similarity_scores[idx]
    print(f"Similarity score: {score:.4f}")
    print(f"Context: {cxt}")
    print()

Top 3 similar context sentences:
Similarity score: 0.6694
Context: I prefer draught over canned or bottled beer.

Similarity score: 0.6656
Context: I like beer. I enjoy the more relaxing setting of a country concert.

Similarity score: 0.6629
Context: I like beer.



In [27]:
def save_dataset(lst, file_path):
    with open(file_path, "w") as file:
        for line in lst:
            string_list = [str(num) for num in line]
            result = ', '.join(string_list)
            file.write(result + "\n")

folder_path = '/content/drive/MyDrive/Colab Notebooks/Metabuddy/Data Process/week2'
file_path = f'{folder_path}/test_embedding.txt'
save_dataset(context_embedding, file_path)

In [ ]:
with open(file_path, "r") as file:
    i=0
    for line in file:
        if i == 10:
            break
        print(line)
        i+=1

In [42]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [43]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [46]:
model_name = "t5-base/checkpoint-3000"
model_dir = f"drive/MyDrive/Colab Notebooks/Metabuddy/Models/week2/{model_name}"
max_input_length = 512

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

In [47]:
prefix = 'summarize: '
input_text = "Hey I do too! In florida for vacation I do it every winter Comedies are good, on thursdays I sing karaoke. No good at tennis, but I was a champion snow skier in college. Mine is yellow. Any other hobbies or likes? I like a good book, but I find I get more into movies. That's an awesome movie. Caddyshack is hilarious too I am suprised you had seen coming to america. I like a lot of bill murray Well hope you have a good rest of your day!"

input = [prefix + input_text]

inputs = tokenizer(input, max_length=max_input_length, truncation=True, return_tensors="pt")
outputs = model.generate(**inputs, max_length=100, num_return_sequences=1, early_stopping=True)
for output in outputs:
    summarized_text = tokenizer.decode(output, skip_special_tokens=True)
    print(summarized_text)

I like to read. I like to sing karaoke. I like to watch movies. I am a champion snow skier. I live in Florida.
